In [1]:
import glob
import numpy             as np
import pandas            as pd
import tables            as tb
import matplotlib.pyplot as plt

from invisible_cities.io.pmaps_io import load_pmaps_as_df

to_df = pd.DataFrame.from_records

In [2]:
dtype = "pmaps"
run_number = 7255

icversion_datadir  = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/v0.9.9/"
testprod_datadir   = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/IC-testprod/"
testprod_p_datadir = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/IC-testprod-p/"

icversion_files  = glob.glob(icversion_datadir  + f"*")
testprod_files   = glob.glob(testprod_datadir   + f"*")
testprod_p_files = glob.glob(testprod_p_datadir + f"*")

icversion_files.sort()
testprod_files.sort()
testprod_p_files.sort()

In [16]:
def loop_in_events(Spmap, SPMTpmap, EVENTS):
    n, time, energy, height, width, rms, most_active_pmt, mact_percentage = [], [], [], [], [], [], [], []
    events = []
    
    for ev in EVENTS:
        S    = Spmap   [Spmap   ["event"]==ev]
        SPMT = SPMTpmap[SPMTpmap["event"]==ev] 
    
        PEAKS = np.unique(S["peak"])
        if len(PEAKS)>0: n.append(np.max(PEAKS)+1)
        else:            n.append(0)
    
        for pk in PEAKS:
            #Total
            times = S[S["peak"]==pk]["time"].values
            enes  = S[S["peak"]==pk]["ene"] .values
            
            events.append(ev)
            time  .append(times[np.argmax(enes)]/10**3)
            height.append(np.max(enes))
            energy.append(np.sum(enes))
            width .append(times[-1]-times[0])
            
            PMTS  = np.unique(SPMT["npmt"])
            esum = []
            for pmt in PMTS:
                enes = SPMT[SPMT["npmt"]==pmt]["ene"].values
                esum.append( np.sum(enes[enes>0]) )
            most_active_pmt.append(PMTS[np.argmax(esum)])
            mact_percentage.append( np.max(esum)/np.sum(esum)*100)
    return events, n, time, height, energy, width, most_active_pmt , mact_percentage   

In [18]:
pmapdir = icversion_datadir
npmaps = 2
outfilename = "./summary_pmtsignal_icversion.h5"

Pmaps_PMTSignal_info(pmapdir, npmaps, outfilename)

In [19]:
pd.read_hdf(outfilename, key="nS").head()

,event,nS1s,nS2s
0,1,4,1
1,2,4,1
2,3,1,1
3,4,3,1
4,5,7,1


In [24]:
pd.read_hdf(outfilename, key="S1").head()

,event,time,height,energy,width,mactPMT,mact_perc
0,1,113.100,0.607815,1.287501,200.0,1,14.567706
1,1,236.625,0.820713,0.942436,200.0,1,14.567706
2,1,295.800,2.510230,13.820708,275.0,1,14.567706
3,1,312.700,0.719642,0.040256,200.0,1,14.567706
4,2,8.350,0.826929,2.181855,175.0,4,15.567245


In [21]:
pd.read_hdf(outfilename, key="S2").head()

,event,time,height,energy,width,mactPMT,mact_perc
0,1,804.475812,1649.783813,10208.359375,12425.3125,1,11.765875
1,2,802.483062,1026.768799,4947.874512,9486.1250,1,11.263434
2,3,802.501875,1831.519165,8676.576172,9712.4375,1,11.913650
3,4,802.481188,2843.324951,8872.907227,7736.3125,1,11.610174
4,5,803.504625,2900.474121,12070.620117,10457.5625,1,11.546741


In [22]:
pd.read_hdf(outfilename, key="nevent").head()

,file,nevent
0,0,272
1,1,265
